#NLP- room service chatbot text classification

#Import Libraries

In [ ]:
import nltk

In [ ]:
!nltk.download_gui()

/bin/bash: -c: line 1: syntax error: unexpected end of file


In [ ]:
nltk.download

<bound method Downloader.download of <nltk.downloader.Downloader object at 0x7f0c98b5ca10>>

In [ ]:
import re
import os
import csv
from nltk.stem.snowball import SnowballStemmer
import random
from nltk.classify import SklearnClassifier
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np
import pandas as pd

In [ ]:
## Get multiple outputs in the same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Ignore all warnings
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

In [ ]:
## Display all rows and columns of a dataframe instead of a truncated version
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Preprocess (Sentence Test)

In [ ]:
# Test
sentence = "The Big brown fox jumped over a lazy dog."
sentence2 = "This is particularly important in today's world where we are swamped with unstructured natural language data on the variety of social media platforms people engage in now-a-days (note -  now-a-days in the decade of 2010-2020)"

In [ ]:
#convert sentence to lower case
'This' == 'this'
print('AbcdEFgH'.lower())
sentence.lower()
sentence2.lower()

False

abcdefgh


'the big brown fox jumped over a lazy dog.'

"this is particularly important in today's world where we are swamped with unstructured natural language data on the variety of social media platforms people engage in now-a-days (note -  now-a-days in the decade of 2010-2020)"

### Tokenize -extract individual words

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
"""tokens = tokenizer.tokenize(sentence)
tokens
tokens2 = tokenizer.tokenize(sentence2)
tokens2"""

'tokens = tokenizer.tokenize(sentence)\ntokens\ntokens2 = tokenizer.tokenize(sentence2)\ntokens2'

### Stopwords : Filter words to remove non-useful words

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
"""
filtered_words = [w for w in tokens if not w in stopwords.words('english')]
filtered_words
"""

['The', 'Big', 'brown', 'fox', 'jumped', 'lazy', 'dog']

In [ ]:
"""
filtered_words = [w for w in tokens2 if not w in stopwords.words('english')]
filtered_words
"""

['This',
 'particularly',
 'important',
 'today',
 'world',
 'swamped',
 'unstructured',
 'natural',
 'language',
 'data',
 'variety',
 'social',
 'media',
 'platforms',
 'people',
 'engage',
 'days',
 'note',
 'days',
 'decade',
 '2010',
 '2020']

In [ ]:
def preprocess(sentence):
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [w for w in tokens if not w in stopwords.words('english')]
    return filtered_words

In [ ]:
"""
preprocessed_sentence = preprocess(sentence)
print(preprocessed_sentence)
"""

['big', 'brown', 'fox', 'jumped', 'lazy', 'dog']


In [ ]:
"""
preprocess(sentence2)
"""

['particularly',
 'important',
 'today',
 'world',
 'swamped',
 'unstructured',
 'natural',
 'language',
 'data',
 'variety',
 'social',
 'media',
 'platforms',
 'people',
 'engage',
 'days',
 'note',
 'days',
 'decade',
 '2010',
 '2020']

## Tagging

POS tag list:

CC coordinating conjunction
CD cardinal digit
DT determiner
EX existential there (like: "there is" ... think of it like "there exists")
FW foreign word
IN preposition/subordinating conjunction
JJ adjective 'big'
JJR adjective, comparative 'bigger'
JJS adjective, superlative 'biggest'
LS list marker 1)
MD modal could, will
NN noun, singular 'desk'
NNS noun plural 'desks'
NNP proper noun, singular 'Harrison'
NNPS proper noun, plural 'Americans'
PDT predeterminer 'all the kids'
POS possessive ending parent's
PRP personal pronoun I, he, she
PRP$ possessive pronoun my, his, hers
RB adverb very, silently,
RBR adverb, comparative better
RBS adverb, superlative best
RP particle give up
TO to go 'to' the store.
UH interjection errrrrrrrm
VB verb, base form take
VBD verb, past tense took
VBG verb, gerund/present participle taking
VBN verb, past participle taken
VBP verb, sing. present, non-3d take
VBZ verb, 3rd person sing. present takes
WDT wh-determiner which
WP wh-pronoun who, what
WP$ possessive wh-pronoun whose
WRB wh-abverb where, when

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
"""
tags = nltk.pos_tag(preprocessed_sentence)
print(tags)
"""

[('big', 'JJ'), ('brown', 'NN'), ('fox', 'NN'), ('jumped', 'VBD'), ('lazy', 'JJ'), ('dog', 'NN')]


In [ ]:
"""
tags = nltk.pos_tag(preprocess(sentence2))
print(tags)
"""

[('particularly', 'RB'), ('important', 'JJ'), ('today', 'NN'), ('world', 'NN'), ('swamped', 'VBD'), ('unstructured', 'JJ'), ('natural', 'JJ'), ('language', 'NN'), ('data', 'NNS'), ('variety', 'NN'), ('social', 'JJ'), ('media', 'NNS'), ('platforms', 'NNS'), ('people', 'NNS'), ('engage', 'VBP'), ('days', 'NNS'), ('note', 'VBP'), ('days', 'NNS'), ('decade', 'NN'), ('2010', 'CD'), ('2020', 'CD')]


## Extracting only Nouns and Verb nouns

In [ ]:
def extract_tagged(sentences):
    features = []
    for tagged_word in sentences:
        word, tag = tagged_word
        if tag=='NN' or tag == 'VBN' or tag == 'NNS' or tag == 'VBP' or tag == 'RB' or tag == 'VBZ' or tag == 'VBG' or tag =='PRP' or tag == 'JJ':
            features.append(word)
    return features

In [ ]:
"""
extract_tagged(tags)
"""

['particularly',
 'important',
 'today',
 'world',
 'unstructured',
 'natural',
 'language',
 'data',
 'variety',
 'social',
 'media',
 'platforms',
 'people',
 'engage',
 'days',
 'note',
 'days',
 'decade']

## Lemmatize words

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [ ]:
lmtzr = WordNetLemmatizer()
"""
print(lmtzr.lemmatize('cacti'))
print(lmtzr.lemmatize('willing'))
print(lmtzr.lemmatize('feet'))
print(lmtzr.lemmatize('stemmed'))

print(lmtzr.lemmatize('cactus'))
"""

"\nprint(lmtzr.lemmatize('cacti'))\nprint(lmtzr.lemmatize('willing'))\nprint(lmtzr.lemmatize('feet'))\nprint(lmtzr.lemmatize('stemmed'))\n\nprint(lmtzr.lemmatize('cactus'))\n"

## Stem words

In [ ]:
# words_for_stemming = ['stem', 'stemming', 'stemmed', 'stemmer', 'stems','feet','willing']

In [ ]:
stemmer = SnowballStemmer("english")
"""[stemmer.stem(x) for x in words_for_stemming]"""

'[stemmer.stem(x) for x in words_for_stemming]'

## Put together

In [ ]:
def extract_feature(text):
    words = preprocess(text)
#     print('words: ',words)
    tags = nltk.pos_tag(words)
#     print('tags: ',tags)
    extracted_features = extract_tagged(tags)
#     print('Extracted features: ',extracted_features)
    stemmed_words = [stemmer.stem(x) for x in extracted_features]
#     print(stemmed_words)

    result = [lmtzr.lemmatize(x) for x in stemmed_words]
   
    return result

In [ ]:
# sentence

'The Big brown fox jumped over a lazy dog.'

In [ ]:
"""
words = extract_feature(sentence)
print(words)
"""

['big', 'brown', 'fox', 'lazi', 'dog']


In [ ]:
"""
words = extract_feature(sentence2)
print(words)
"""

['particular', 'import', 'today', 'world', 'unstructur', 'natur', 'languag', 'data', 'varieti', 'social', 'medium', 'platform', 'peopl', 'engag', 'day', 'note', 'day', 'decad']


In [ ]:
# extract_feature("He hurt his right foot while he was wearing white shoes on his feet")

['hurt', 'right', 'foot', 'wear', 'white', 'shoe', 'foot']

## Bag of words

In [ ]:
def word_feats(words):
    return dict([(word, True) for word in words])

In [ ]:
# word_feats(words)

NameError: ignored

##Parsing

In [ ]:
def extract_feature_from_doc(data):
    result = []
    corpus = []
    # The responses of the chat bot
    answers = {}
    for (text,category,answer) in data:

        features = extract_feature(text)

        corpus.append(features)
        result.append((word_feats(features), category))
        answers[category] = answer  # assign [key]= value

    return (result, sum(corpus,[]), answers)

In [ ]:
# Test function
extract_feature_from_doc([['this is the input text from the user','category','answer to give']])

([({'input': True, 'user': True}, 'category')],
 ['input', 'user'],
 {'category': 'answer to give'})

In [ ]:
def get_content(filename):
    doc = os.path.join(filename)
    with open(doc, 'r') as content_file:
        lines = csv.reader(content_file,delimiter='|')
        data = [x for x in lines if len(x) == 3] #Read data from array
        return data

In [ ]:
filename = '/content/Guest.txt'
data = get_content(filename)

In [ ]:
data

[['Hello', 'Greetings', 'Hello. I am Dexter. I will serve your requests.'],
 ['hi hello', 'Greetings', 'Hello. I am Dexter. I will serve your requests.'],
 ['hi ', 'Greetings', 'Hello. I am Dexter. I will serve your requests.'],
 ['hi', 'Greetings', 'Hello. I am Dexter. I will serve your requests.'],
 ['hi', 'Greetings', 'Hello. I am Dexter. I will serve your requests.'],
 ['hey', 'Greetings', 'Hello. I am Dexter. I will serve your requests.'],
 ['hello, hi', 'Greetings', 'Hello. I am Dexter. I will serve your requests.'],
 ['hey', 'Greetings', 'Hello. I am Dexter. I will serve your requests.'],
 ['hey, hi', 'Greetings', 'Hello. I am Dexter. I will serve your requests.'],
 ['hey, hello',
  'Greetings',
  'Hello. I am Dexter. I will serve your requests.'],
 ['Good morning',
  'Morning',
  'Good Morning. I am Dexter. I will serve your requests.'],
 ['Good afternoon',
  'Afternoon',
  'Good afternoon. I am Dexter. I will serve your requests.'],
 ['Good evening',
  'Evening',
  'Good eveni

In [ ]:
features_data, corpus, answers = extract_feature_from_doc(data)

In [ ]:
print(features_data[40]) #check the value

({'order': True, 'taken': True}, 'Utilized-Order')


In [ ]:
corpus

['hello',
 'hi',
 'hello',
 'hi',
 'hi',
 'hi',
 'hey',
 'hello',
 'hi',
 'hey',
 'hey',
 'hi',
 'hey',
 'hello',
 'good',
 'morn',
 'good',
 'afternoon',
 'good',
 'even',
 'good',
 'night',
 'today',
 'want',
 'help',
 'need',
 'help',
 'help',
 'want',
 'help',
 'want',
 'assist',
 'help',
 'great',
 'talk',
 'great',
 'thank',
 'help',
 'thank',
 'thank',
 'much',
 'thank',
 'thank',
 'much',
 'mani',
 'type',
 'food',
 'type',
 'food',
 'type',
 'food',
 'food',
 'type',
 'mani',
 'order',
 'take',
 'mani',
 'order',
 'take',
 'mani',
 'order',
 'take',
 'mani',
 'order',
 'taken',
 'mani',
 'order',
 'alreadi',
 'taken',
 'tell',
 'order',
 'count',
 'taken',
 'mani',
 'order',
 'taken',
 'number',
 'order',
 'taken',
 'order',
 'taken',
 'number',
 'order',
 'alreadi',
 'taken',
 'order',
 'alreadi',
 'taken',
 'tell',
 'number',
 'order',
 'taken',
 'taken',
 'order',
 'order',
 'taken',
 'mani',
 'special',
 'order',
 'taken',
 'mani',
 'special',
 'order',
 'special',
 'order

In [ ]:
answers

{'Afternoon': 'Good afternoon. I am Dexter. I will serve your requests.',
 'Closing': "It's glad to know that I have been helpful. Have a good day!",
 'Default-Option': 'You have 25 options.',
 'Default-Utilized-Introduction': 'You can order 12.',
 'Evening': 'Good evening. I am Dexter. I will serve your requests.',
 'Food-Type': 'Currently I know about two: Italian and British food.',
 'Goodbye': 'Good night. I am off. Take care.',
 'Greetings': 'Hello. I am Dexter. I will serve your requests.',
 'Help': 'How can I help you?',
 'LR': 'You have 30 living records.',
 'Morning': 'Good Morning. I am Dexter. I will serve your requests.',
 'No-Help': 'Ok sir/madam. No problem. Have a nice day.',
 'Opening': "I'm fine! Thank you. How can I help you?",
 'Option': 'You have 5 options remaining.',
 'Special-Option': 'You have 2 special options remaining.',
 'Utilized-Order': 'You have ordered 6.',
 'Utilized-Special-Order': 'You have taken 2 special orders.'}

# Train the model by using these features

In [ ]:
# split data into train and test sets
split_ratio = 0.8

In [ ]:
def split_dataset(data, split_ratio):
    random.shuffle(data)
    data_length = len(data)
    train_split = int(data_length * split_ratio)
    return (data[:train_split]), (data[train_split:])

In [ ]:
training_data, test_data = split_dataset(features_data, split_ratio)

In [ ]:
training_data # check training data

[({'count': True, 'option': True, 'remain': True, 'tell': True}, 'Option'),
 ({'option': True, 'remain': True}, 'Option'),
 ({'mani': True, 'option': True, 'remain': True}, 'Default-Option'),
 ({'number': True, 'order': True, 'special': True, 'taken': True},
  'Utilized-Special-Order'),
 ({'number': True, 'order': True, 'special': True, 'taken': True},
  'Utilized-Special-Order'),
 ({'count': True, 'option': True, 'remain': True, 'special': True},
  'Special-Option'),
 ({'mani': True, 'order': True, 'taken': True}, 'Utilized-Order'),
 ({'hey': True, 'hi': True}, 'Greetings'),
 ({'number': True, 'option': True, 'remain': True, 'special': True},
  'Special-Option'),
 ({'great': True}, 'Closing'),
 ({'live': True, 'number': True, 'record': True}, 'LR'),
 ({'mani': True, 'order': True, 'special': True, 'taken': True},
  'Utilized-Special-Order'),
 ({'count': True, 'live': True, 'record': True, 'tell': True}, 'LR'),
 ({'balanc': True, 'option': True, 'special': True}, 'Special-Option'),
 ({

In [ ]:
# save the data
np.save('training_data', training_data)
np.save('test_data', test_data)

## Decision Tree

In [ ]:
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [ ]:
training_data = np.load('training_data.npy')
test_data = np.load('test_data.npy')

In [ ]:
def train_using_decision_tree(training_data, test_data):
    
    classifier = nltk.classify.DecisionTreeClassifier.train(training_data, entropy_cutoff=0.6, support_cutoff=6)
    classifier_name = type(classifier).__name__
    training_set_accuracy = nltk.classify.accuracy(classifier, training_data)
    print('training set accuracy: ', training_set_accuracy)
    test_set_accuracy = nltk.classify.accuracy(classifier, test_data)
    print('test set accuracy: ', test_set_accuracy)
    return classifier, classifier_name, test_set_accuracy, training_set_accuracy

In [ ]:
dtclassifier, classifier_name, test_set_accuracy, training_set_accuracy = train_using_decision_tree(training_data, test_data)

training set accuracy:  0.9047619047619048
test set accuracy:  0.8518518518518519


## Naive Bayes

In [ ]:
def train_using_naive_bayes(training_data, test_data):
    classifier = nltk.NaiveBayesClassifier.train(training_data)
    classifier_name = type(classifier).__name__
    training_set_accuracy = nltk.classify.accuracy(classifier, training_data)
    test_set_accuracy = nltk.classify.accuracy(classifier, test_data)
    return classifier, classifier_name, test_set_accuracy, training_set_accuracy

In [ ]:
classifier, classifier_name, test_set_accuracy, training_set_accuracy = train_using_naive_bayes(training_data, test_data)
print(training_set_accuracy)
print(test_set_accuracy)
print(len(classifier.most_informative_features()))
classifier.show_most_informative_features()

0.819047619047619
0.7777777777777778
64
Most Informative Features
                 special = None           Option : Specia =     14.3 : 1.0
                   taken = None           Specia : Utiliz =      6.6 : 1.0
                    take = True           Defaul : Specia =      6.6 : 1.0
                    mani = True           Defaul : Option =      6.0 : 1.0
                 alreadi = True           Utiliz : Utiliz =      4.0 : 1.0
                  remain = None           Utiliz : Option =      3.5 : 1.0
                      hi = None           Specia : Greeti =      2.7 : 1.0
                   thank = None           Specia : Closin =      2.6 : 1.0
                    tell = True               LR : Defaul =      2.5 : 1.0
                   count = True           Utiliz : LR     =      2.4 : 1.0


In [ ]:
classifier.classify(({'take': True, 'count': True, 'remain': True}))

'Special-Option'

In [ ]:
extract_feature("hello")

['hello']

In [ ]:
word_feats(extract_feature("hello"))

{'hello': True}

## Test performance

In [ ]:
## Category classification
input_sentence = "How many special orders have I taken"
classifier.classify(word_feats(extract_feature(input_sentence)))

'Utilized-Special-Order'

In [ ]:
def reply(input_sentence):
  category = dtclassifier.classify(word_feats(extract_feature(input_sentence)))
  return answers[category]

In [ ]:
reply('thank you for your help')

"It's glad to know that I have been helpful. Have a good day!"

In [ ]:
reply('special orders taken?')

'You have taken 2 special orders.'